In [1]:
import pickle
import sys

In [3]:
data = pickle.load(open('student_answer.pkl','rb'))
hint_data = pickle.load(open('student_hint.pkl','rb'))

In [4]:
attempt = {}
for line in data:
    if line['timestamp'] == 'null':
        continue
    answer = [line['timestamp'],line['attempt'],line['score']]
    key = (line['set_id'],line['problem_id'],line['part_id'])
    if key not in attempt:
        attempt[key] = {}
    student_key = (str(line['user_id']).replace('L',''),line['username'])
    if student_key not in attempt[key]:
        attempt[key][student_key] = []
    attempt[key][student_key].append(answer)

In [5]:
### Run this to see the number of wrong attempts for each week each problem and each part.
print('Total attempt number for each problem part\n')
for problem in sorted(attempt):
    total_attempt_number = 0
    for student in attempt[problem]:
        for ans in attempt[problem][student]:
            if ans[2] == '0':
                total_attempt_number += 1
            elif ans[2] == '1' or ans[2] == '2':
                break
    print('week:' + problem[0] + ' problem:' + problem[1] + ' part:' + problem[2] + ' total attempts:' +str(total_attempt_number)) + '\n'

Total attempt number for each problem part

week:2 problem:10 part:1 total attempts:11

week:2 problem:10 part:2 total attempts:13

week:2 problem:10 part:3 total attempts:29

week:2 problem:10 part:4 total attempts:19

week:2 problem:11 part:1 total attempts:14

week:2 problem:11 part:2 total attempts:24

week:2 problem:12 part:1 total attempts:105

week:2 problem:12 part:2 total attempts:126

week:2 problem:12 part:3 total attempts:132

week:2 problem:12 part:4 total attempts:156

week:2 problem:12 part:5 total attempts:234

week:2 problem:12 part:6 total attempts:277

week:2 problem:13 part:1 total attempts:77

week:2 problem:13 part:2 total attempts:51

week:2 problem:13 part:3 total attempts:189

week:2 problem:5 part:1 total attempts:60

week:2 problem:5 part:2 total attempts:30

week:2 problem:6 part:1 total attempts:41

week:2 problem:6 part:2 total attempts:57

week:2 problem:7 part:1 total attempts:284

week:2 problem:7 part:2 total attempts:178

week:2 problem:7 part:3 total

In [15]:
### Select certain week_id, problem_id, and part_id as int to show below

### Type in integer for the week id
week_id = 4

### If you want to see all problems of certain week, put problem_usr = 0
problem_id = 9

### If you want to see all parts of certain problem,  put part_usr = 0
part_id = 3

In [11]:
for problem in attempt:
    id = 1
    if (week_id != 0 and problem[0] != str(week_id)) or \
    (problem_id != 0 and problem[1] != str(problem_id)) or \
    (part_id != 0 and problem[2] != str(part_id)):
        continue 
    print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
    print problem
    for student in attempt[problem]:
        print('ID:' + str(id) +' ---------------------------------------' +  ' week:'
              + problem[0] + ' problem:' + problem[1] + ' part:' + problem[2])
        id += 1
        print(student)
        for ans in attempt[problem][student]:
            if ans[1] == 'null' or ans[1] == '' or ans[2] == 'null':
                continue
            print(ans)

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
('4', '3', '3')
ID:1 --------------------------------------- week:4 problem:3 part:3
('61', 'ThucPhan')
[u'2016-10-18 20:47:51', u'(92/100)*0.0001*100', '0']
[u'2016-10-18 20:52:22', u'(94/100)*(0.0001)*(100)', '0']
[u'2016-10-18 21:22:15', u'(93/100)*0.0001', '1']
ID:2 --------------------------------------- week:4 problem:3 part:3
('289', 'maxcjx')
[u'2016-10-17 18:30:59', u'0.95*(1/10000)', '1']
ID:3 --------------------------------------- week:4 problem:3 part:3
('153', 'CheyennePayne')
[u'2016-10-18 07:07:24', u'0.0001*((0.94*0.0001)+(0.0199.9999))', '0']
[u'2016-10-18 07:16:58', u'0.92*0.0001', '1']
ID:4 --------------------------------------- week:4 problem:3 part:3
('118', 'alebedev')
[u'2016-10-16 21:21:04', u'0.000095', '1']
ID:5 --------------------------------------- week:4 problem:3 part:3
('57', 'jar045')
[u'2016-10-19 05:44:57', u'(92/100)*(1/10000) * 100', '0']
[u'2016-10-19 05:47:14', u'0.95*0.0001*100', '0']
[u'2016-10-19 05

In [16]:
#combine two dic
for problem in attempt:
    for student in attempt[problem]:
        if problem in hint_data and student in hint_data[problem]:
            for hint_content in hint_data[problem][student]:
                attempt[problem][student].append(hint_content)
            attempt[problem][student].sort()

for problem in attempt:
    id = 1 
    if (week_id != 0 and problem[0] != str(week_id)) or \
    (problem_id != 0 and problem[1] != str(problem_id)) or \
    (part_id != 0 and problem[2] != str(part_id)):
        continue 
    print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
    print problem
    for student in attempt[problem]:
        print('No:' + str(id) +'---------------------------------------' +  ' week:' + problem[0] + ' problem:' + problem[1] + ' part:' + problem[2])
        id += 1
        print(student)
        for ans in attempt[problem][student]:
            if ans[1] == 'null' or ans[1] == '' or ans[2] == 'null':
                continue
            print(ans)

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
('4', '9', '3')
No:1--------------------------------------- week:4 problem:9 part:3
('114', 'QIQIWU')
[u'2016-10-18 01:19:06', u'12*13', '1']
[u'2016-10-18 02:41:44', u'13*12', '0']
[u'2016-10-18 02:44:45', u'12*11', '1']
No:2--------------------------------------- week:4 problem:9 part:3
('61', 'ThucPhan')
[u'2016-10-19 02:54:37', u'3', '0']
[u'2016-10-19 02:59:48', u'C(15,3)', '0']
[u'2016-10-19 03:02:25', u'(14-1)*(14-2)', '1']
No:3--------------------------------------- week:4 problem:9 part:3
('232', 'RobinO')
[u'2016-10-18 03:50:40', u'13*12', '1']
[u'2016-10-18 03:53:51', u'12*11', '1']
No:4--------------------------------------- week:4 problem:9 part:3
('215', 'Zhongting')
[u'2016-10-16 07:30:34', u'13*12', '1']
No:5--------------------------------------- week:4 problem:9 part:3
('289', 'maxcjx')
[u'2016-10-18 04:33:04', u'12*11', '1']
No:6--------------------------------------- week:4 problem:9 part:3
('321', 'Sync')
[u'2016-10-19 00